In [1]:
from pyspark.sql import SparkSession

def _spark_context():
    'Creates a local spark context'

    return SparkSession.builder \
      .master('local') \
      .appName('syllabus') \
      .getOrCreate()

SPARK = _spark_context()
SPARK

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/21 00:16:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [29]:
import json

from pyspark.sql import DataFrame
from pyspark.sql import functions as F

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalTrueColorFormatter

def ppj(j, indent=2):
    j = json.dumps(json.loads(j), indent=indent)
    print(highlight(j, JsonLexer(), TerminalTrueColorFormatter()))
   
def ppd(d, indent=2):
    ppj(json.dumps(d, indent=indent))

def count_nulls(df: DataFrame) -> int:
    return df.select(
        sum([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
    ).collect()[0][0]

def count_cells(df: DataFrame) -> int:
    return df.count() * len(df.columns)

class DFLoader:
    @staticmethod
    def from_file(records: list, fpath: str) -> DataFrame:
        with open(fpath, 'w') as ostream:
            for record in records:
                print(json.dumps(record), file=ostream, end='\n')
        df = SPARK.read.json(fpath)
        df.show()
        print('cells', count_cells(df), '/', 'nulls', count_nulls(df))
        print(ppj(df.schema.json()))
        return df    


In [30]:
records = [
    { "a": "a", "c": "d" },
    { "a": "b" },
    { "a": "c" },
    { "a": 1, "d": "z" }
]
df = DFLoader.from_file(records, 'f.ndjson')
count_nulls(df)

+---+----+----+
|  a|   c|   d|
+---+----+----+
|  a|   d|null|
|  b|null|null|
|  c|null|null|
|  1|null|   z|
+---+----+----+

cells 12 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}

None


6

In [31]:
records = [
    { "a": 1, "c": "d" },
    { "a": "b" },
    { "a": "c" },
    { "a": "123", "d": "z" }
]
df = DFLoader.from_file(records, 'f.ndjson')

+---+----+----+
|  a|   c|   d|
+---+----+----+
|  1|   d|null|
|  b|null|null|
|  c|null|null|
|123|null|   z|
+---+----+----+

cells 12 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}

None


In [32]:
records = [
    { "id": 123, "a": 1, "c": "d" },
    { "id": 122, "a": "b" },
    { "id": 111, "a": "c" },
    { "id": 234, "a": 1, "d": "z" }
]
df = DFLoader.from_file(records, 'f.ndjson')

+---+----+----+---+
|  a|   c|   d| id|
+---+----+----+---+
|  1|   d|null|123|
|  b|null|null|122|
|  c|null|null|111|
|  1|null|   z|234|
+---+----+----+---+

cells 16 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}

None


In [33]:
records = [
    { "id": 123, "key": "a", "value": 1},
    { "id": 123, "key": "c", "value": "d" },
    { "id": 122, "key": "a", "value": "b" },
    { "id": 111, "key": "a", "value": "c" },
    { "id": 234, "key": "a", "value": 1 },
    { "id": 234, "key": "d", "value": "z" },
]
df = DFLoader.from_file(records, 'f.ndjson')

+---+---+-----+
| id|key|value|
+---+---+-----+
|123|  a|    1|
|123|  c|    d|
|122|  a|    b|
|111|  a|    c|
|234|  a|    1|
|234|  d|    z|
+---+---+-----+

cells 18 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    },
    {
      "metadata": {},
      "name": "key",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "value",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}

None
